In [ ]:
%pip install langchain qdrant_client openai tiktoken

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 12.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 182.2/182.2 kB 13.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 220.8/220.8 kB 18.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 28.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.8/46.8 kB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 39.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.0/75.0 kB 12.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.8/143.8 kB 26.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.6/294.6 kB 44.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.3/5.3 MB 61.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.9/76.9 kB 14.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
%pip install PyPDF2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 4.7 MB/s eta 0:00:00


In [ ]:
%pip install sentence-transformers

In [ ]:
from PyPDF2 import PdfReader
from langchain.vectorstores import Qdrant
from langchain.embeddings.openai import OpenAIEmbeddings
from qdrant_client.http.models import PointStruct
from qdrant_client import QdrantClient,models
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import Qdrant
from sentence_transformers import SentenceTransformer
from langchain.chat_models import ChatOpenAI
from langchain.prompts.prompt import PromptTemplate
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationalRetrievalChain, RetrievalQA
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.callbacks import get_openai_callback
from langchain.vectorstores import DeepLake
from langchain.chains.question_answering import load_qa_chain
from langchain.prompts.chat import (
    ChatPromptTemplate,
    MessagesPlaceholder,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
)
from langchain.chains import LLMChain
import PyPDF2
from langchain.document_loaders import PyPDFLoader
import glob
import time
import qdrant_client
import os
import uuid
import openai

In [ ]:
# create a qdrant_client

# qdrant Token

client = qdrant_client.QdrantClient(
    os.getenv("QDRANT_HOST"),
    api_key=os.getenv("QDRANT_API_KEY"),
)

In [ ]:
# Create collection

os.environ['QDRANT_COLLECTION_NAME'] = "my-collections-test" #cambia esto acorde a tu base de datos de vectores(si ya tienes no es necesario crear un lector de pdf)
                                                            # Change this according to your vector database (if you already have one, there is no need to create a PDF reader)
vectors_config = qdrant_client.http.models.VectorParams(
    size=1536,
    distance=qdrant_client.http.models.Distance.COSINE
)

client.create_collection(
    collection_name=os.getenv("QDRANT_COLLECTION_NAME"),
    vectors_config=vectors_config,
)


In [ ]:
# Vector store.

os.environ['OPENAI_API_KEY'] = "PUT YOUR TOKEN HERE"

embeddings = OpenAIEmbeddings()
# knowlegde_base = QdrantClient.text

vector_store = Qdrant(
    client=client,
    collection_name=os.getenv("QDRANT_COLLECTION_NAME"),
    embeddings=embeddings,
)


In [ ]:
knowledge_base = Chroma.from_documents(chunks, embeddings, persist_directory = persist_directory)
new_knowledge_base = Qdrant(persist_directory = persist_directory, embedding_function = embeddings)

In [ ]:
# Add document to vector store.  Esto es para leer el pdf, pasarlo a chunks y meterlo en los vector store

text_splitter = CharacterTextSplitter(
    chunk_size = 800,
    chunk_overlap = 100,
    length_function = len
)

pdf_file_obj = open('habitos-atomicos-james-clear.pdf', 'rb')
pdf_reader = PdfReader(pdf_file_obj)

text = ""
for page in pdf_reader.pages:
    text += page.extract_text()

chunks = text_splitter.split_text(text)

'''
texts = get_chunks(pdf_reader)
'''

vector_store.add_texts(chunks)


['d71379a5977d4ae3ae3ae5fc29bc3d94']

In [ ]:
knowledge_base = Qdrant.from_documents(chunks, embeddings, persist_directory = persist_directory)

knowledge_base.persist()

new_knowledge_base = Qdrant(persist_directory = persist_directory, embedding_function = embeddings)

In [ ]:
# retrieval chain. Esto es para llamar. Si ya tienes una base de datos con los vectores no es necesario crear una funcion que lea, sencillamente recarga
# Lo que quiero es meter esto osea la variable qa dentro de un texto
from langchain.chains import RetrievalQA
from langchain.llms import OpenAI

qa = RetrievalQA.from_chain_type(
    llm=ChatOpenAI(temperature = 0,
                   model_name='gpt-3.5-turbo', callbacks=[StreamingStdOutCallbackHandler()], streaming = True,)
    #retriever=vector_store.as_retriever()
)

In [ ]:
# Test de la base de datos vectorial
# Test of Vectorial DB

query = "Como hago un buen habito?"

response = qa.run(query)

print(response)

In [ ]:
from langchain import LLMChain
from langchain.llms import OpenAI # no uso (don't need it)
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import LLMChainExtractor
from langchain.prompts import PromptTemplate

# Test final

In [ ]:
file_name = "habitos-atomicos-james-clear.pdf"

def print_letter_by_letter(text):
    for char in text:
        print(char, end='', flush=True)
        time.sleep(0.02)

# Test fail

In [ ]:
from langchain.chains import LLMChain
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
from langchain.memory import ConversationBufferMemory


In [ ]:
# code for making texto into chunks

def get_text_chunks(text):
    text_splitter = CharacterTextSplitter(
        separator="\n",
        chunk_size=1000,
        chunk_overlap=200,
        length_function=len
    )
    chunks = text_splitter.split_text(text)
    return chunks

In [ ]:
# chunks to embeddings

def get_embedding(text_chunks, model_id="text-embedding-ada-002"):
    points = []
    for idx, chunk in enumerate(text_chunks):
        response = openai.Embedding.create(
            input=chunk,
            model=model_id
        )
        embeddings = response['data'][0]['embeddings']
        point_id = str(uuid.uuid4()) #esto es para generar un id unico para cada chunk
        points.append(PointStruct(id=point_id, vector=embeddings, payload={"text": chunk}))
        return points

In [ ]:
# Para meter los datos en Qdrant

def insert_data(get_points):
    operation_info = connection.upsert(
    collection_name="my-collections-test",
    wait = True,
    points=get_points
)

In [ ]:
# Creando la pregunta con el contexto
def create_answer_with_context(query):
    response = openai.Embedding.create(
        input=query,

        model="text-embedding-ada-002"
    )
    embeddings = response['data'][0]['embeddings']

    search_result= connection.search(
        collection_name="my-collections-test",
        query_vector=embeddings,
        limit=1
    )
    prompt = "Context:\n"
    for result in search_result:
        prompt += result.payload['text'] + "\n---\n"
    prompt += "Question" + query + "\n---\n" + "Answer:"

    print("---- PROMPT START ----")
    print(":", prompt)
    print("---- PROMPT END ----")

    completion = openai.ChatCompletion.create(
        model = "gpt-3.5-turbo",
        messages =[
            {"role": "user", "content": prompt}
        ]
    )
    return completion.choice[0].messege.content

In [ ]:
def main():
    get_raw_text= read_data_from_pdf()
    chunks= get_text_chunks(get_raw_text)
    vectors= get_embedding(chunks)

    insert_data(vectors)
    question="Dime como crear buenos habitos"
    answer=create_answer_with_context(question)
    print(answer)

In [ ]:
if __name__ == '__main__':
    main()

NameError: ignored